# Coursera Capstone Week 3

### Necessary Packages to be imported

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import folium 
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

## Part1

### Getting the Data Frame from Wikipedia

In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'class' : 'wikitable sortable'})
rows = table.find_all('tr')
columns = ['Postal Code', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = columns)
for i in range(1, len(rows)):
    row = rows[i].text.split('\n')[1:6:2]
    df.loc[i] = row
df.head()

,Postal Code,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Removing all the Entries with no Borough

In [3]:
df_processed = df[df['Borough'] != 'Not assigned']
df_processed = df_processed.reset_index()
df_processed = df_processed.drop(columns = ['index'])
df_processed.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Checking if there are any Entries left with Not Assigned Neighborhood

In [4]:
dfNoNbhd = df_processed[df_processed['Neighborhood'] == 'Not assigned']
len(dfNoNbhd)

0

## Part2

### Reading a CSV file for Latitudes and Longitudes

In [5]:
df2 = pd.read_csv('https://cocl.us/Geospatial_data')
df_processed2 = df_processed.set_index('Postal Code')
df2 = df2.set_index('Postal Code')
df_processed2['Latitude'] = df2['Latitude']
df_processed2['Longitude'] = df2['Longitude']
df_processed2 = df_processed2.reset_index()
df_processed2.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part3

### Authentication Information

In [15]:
CLIENT_ID = 'QG11RSZ4WFZK203ONXXRESILIC1EUAWKRA4GTF25VZBH3WPG' # your Foursquare ID
CLIENT_SECRET = 'I3X0AQJGB32S0YS0HF4FGH5R2LI2RLTKL4WLQWOEWDK3DP53' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500
latitude = 43.753259
longitude = -79.329656

### Function for getting nearby venues for any location

In [7]:
def getNearbyVenues(names, latitudes, longitudes, postal_codes, boroughs, radius=500):
    
    venues_list=[]
    for name, lat, lng, postal_code, borough in zip(names, latitudes, longitudes, postal_codes, boroughs):
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = None
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
            print(name)
        except :
            pass
        
        venues_list.append([(
            postal_code,
            name,
            borough,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code','Neighborhood', 'Borough', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Executing the Function

In [8]:
toronto_venues = getNearbyVenues(df_processed2['Neighborhood'], df_processed2['Latitude'], df_processed2['Longitude'], df_processed2['Postal Code'], df_processed2['Borough'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

### Checking the Data Frame

In [9]:
toronto_venues.head()

,Postal Code,Neighborhood,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,North York,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,M3A,Parkwoods,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,Victoria Village,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,Victoria Village,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


### One Hot Encoding

In [10]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
toronto_onehot['Postal Code'] = toronto_venues['Postal Code']
toronto_onehot['Borough'] = toronto_venues['Borough']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Postal Code
0,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M3A
1,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M3A
2,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M3A
3,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M4A
4,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M4A


### Grouping the differnent Venues to get characterstics about a Neighborhood

In [11]:
toronto_grouped = toronto_onehot.groupby(['Neighborhood', 'Postal Code', 'Borough']).mean().reset_index()
toronto_grouped.head()

,Neighborhood,Postal Code,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,M1S,Scarborough,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",M8W,Etobicoke,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",M3H,North York,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,M2K,North York,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",M5M,North York,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### K - Means

In [12]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(['Neighborhood','Borough', 'Postal Code'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[1:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Adding Cluster Variable Assigned to the Data Frame

In [13]:
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_grouped

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhood"], 1).set_index("Postal Code"), on="Postal Code")
toronto_merged = toronto_merged[['Postal Code', 'Neighborhood', 'Borough', 'Cluster Labels']]
toronto_merged = toronto_merged.join(df_processed2.drop(["Borough", "Neighborhood"], 1).set_index('Postal Code'), on = 'Postal Code')

toronto_merged.head() # check the last columns!

,Postal Code,Neighborhood,Borough,Cluster Labels,Latitude,Longitude
0,M1S,Agincourt,Scarborough,0,43.794200,-79.262029
1,M8W,"Alderwood, Long Branch",Etobicoke,0,43.602414,-79.543484
2,M3H,"Bathurst Manor, Wilson Heights, Downsview North",North York,0,43.754328,-79.442259
3,M2K,Bayview Village,North York,0,43.786947,-79.385975
4,M5M,"Bedford Park, Lawrence Manor East",North York,0,43.733283,-79.419750


### Map Visualization

In [16]:
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters